In [1]:
import pandas as pd
from tqdm import tqdm

from nltk.corpus import stopwords
import re
from unidecode import unidecode

In [2]:
tqdm.pandas()

In [3]:
tweets = pd.read_csv('tweets_2022_abril_junio.csv')

#### Limpieza de datos

Se filtran Tweets con el mismo id, quedando en el dataframe la primera aparición.

In [4]:
tweets = tweets.drop_duplicates(subset=['id'], keep='first')

Se filtran las columnas que no son de interes.

In [5]:
tweets = tweets.drop(columns=['created_at', 'favorite_count', 'retweet_count'])

#### Procesamiento de Tweets

Se preprocesan los tweets, con el objetivo de estandarizar palabras.

In [6]:
# Lista de stop words en español
stop_words = set(stopwords.words('spanish'))

def preprocess_tweet(tweet):
    # Quitar las tildes y convertir las letras a minúsculas
    tweet = unidecode(tweet).lower()
    
    # Eliminar enlaces
    tweet = re.sub(r'(http\S+)', '', tweet)
    
    # Eliminar todos los caracteres que no sean alfanuméricos o espacios
    tweet = re.sub(r'[^\w\s]', '', tweet)
    
    # Tokenizar el tweet y eliminar stop words y palabras de menos de 3 caracteres
    tweet = ' '.join([token for token in tweet.split() if token not in stop_words and len(token) > 2])
    
    return tweet

In [7]:
processed_tweets = tweets.copy()

# Aplicar la función de preprocesamiento
processed_tweets['text'] = processed_tweets['text'].progress_apply(preprocess_tweet)

100%|██████████████████████████████| 4592806/4592806 [01:26<00:00, 53310.55it/s]


In [8]:
tweet_example = "@usuario Me encanta la #Naturaleza! Visita https://miweb.com. El sol es brillante y el cielo azul. #AireLibre"

preprocess_tweet(tweet_example)

'usuario encanta naturaleza visita sol brillante cielo azul airelibre'

#### Shingles

In [9]:
def calculate_all_shingles(texts, k=3):
    all_shingles = {}
    unique_id = 1

    # Generar los shingles
    for text in tqdm(texts, desc='Calculando shingles', unit='texto'):
        for i in range(len(text) - k + 1):
            shingle = text[i:i + k]
            if shingle not in all_shingles:
                all_shingles[shingle] = unique_id
                unique_id += 1

    return all_shingles

def get_k_shingles(s, k=3):
    """Genera los k-shingles de una cadena de texto"""
    return set([s[i:i+k] for i in range(len(s) - k + 1)])

#### Similitud de Jaccard

In [10]:
def jaccard_similarity(set1, set2):
    """Computa la similitud de Jaccard entre dos sets"""
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) != 0 else 0

## Proceso de exploración para obtener k y s

Tomamos una muestra de 1.000 datos, para ver cuales son los valores de k y s mas optimos, considerando resultado de similitud.

In [11]:
tweets_sample = processed_tweets.sample(n=1_000, random_state=10)

In [12]:
from itertools import combinations

# Función para encontrar los pares de tweets en percentiles específicos
def find_tweet_pairs(df, k, s):
    
    # Calcular los shingles para cada tweet en el DataFrame
    df['shingles'] = df['text'].apply(lambda x: get_k_shingles(x, k))

    # Calcular la similitud de Jaccard entre todos los pares de tweets en el DataFrame
    pairs = list(combinations(df.index, 2))
    total_pairs = len(pairs)
    similarities = []
    with tqdm(total=total_pairs, desc='Calculando similitud', unit='par') as pbar:
        for pair in pairs:
            tweet1 = df.loc[pair[0]]
            tweet2 = df.loc[pair[1]]
            
            # Verificar si los nombres de usuario son diferentes
            if tweet1['screen_name'] != tweet2['screen_name']:
                similarity = jaccard_similarity(tweet1['shingles'], tweet2['shingles'])
                
                # Filtrar los pares de tweets basados en la similitud de Jaccard
                if similarity >= s:
                    similarities.append((pair[0], pair[1], similarity))
                
            pbar.update(1)

    # Ordenar los pares de tweets por similitud
    similarities.sort(key=lambda x: x[2])

    # Obtener los tweets correspondientes a los pares en los percentiles específicos
    percentiles = [0, 25, 50, 75, 95]
    results = {}
    for percentile in percentiles:
        index = int(len(similarities) * percentile / 100)
        pair = similarities[index]
        tweet1 = df.loc[pair[0]]
        tweet2 = df.loc[pair[1]]
        results[percentile] = (tweet1['text'], tweet2['text'])

    # Imprimir los resultados
    for percentile, pair in results.items():
        print(f"Percentil {percentile}:")
        print(f"Tweet 1: {pair[0]}")
        print(f"Tweet 2: {pair[1]}")
        print("------------------------------")

    # Imprimir la cantidad de pares similares y el porcentaje respecto al total de pares
    num_similar_pairs = len(similarities)
    similarity_percentage = num_similar_pairs / total_pairs * 100
    print(f"Cantidad de pares similares (s >= {s}): {num_similar_pairs} ({similarity_percentage:.2f}%)")

#### k = 2

In [13]:
find_tweet_pairs(tweets_sample, 2, 0.3)

Calculando similitud: 100%|█████████| 499500/499500 [00:24<00:00, 20642.97par/s]

Percentil 0:
Tweet 1: angeldlacruz_ alerta confirma convencion actual gobierno boric izquierda quieren expropiar robar ahorros
Tweet 2: danielverdessi despues escuchar convencional marcos barraza cooperativa queda claro propuesta politica impuls
------------------------------
Percentil 25:
Tweet 1: mister_wolf_0 rocio fake cantuarias cerrar debate convencional despacha serie mentiras derechos
Tweet 2: bsepulvedahales derecha quiere apoyar domingos dias habiles trabajo constituyente argumentos
------------------------------
Percentil 50:
Tweet 1: stelartoqui pepe_auth ignacioachurra mineria dano naturaleza banco central autonomo intereses elitistas derecho propi
Tweet 2: nachomaturana gracias valiente incansablemente peleando chile tere_marinovic rocicantuarias arancibial
------------------------------
Percentil 75:
Tweet 1: retirototalafp tere_marinovic explica paso paso expropiaran ahorros quintoretirourgente retirototalafp qui
Tweet 2: tere_marinovic gravisimo constituyentes acaban a

#### k=3

In [14]:
find_tweet_pairs(tweets_sample, 3, 0.1)

Calculando similitud: 100%|█████████| 499500/499500 [00:24<00:00, 20269.40par/s]

Percentil 0:
Tweet 1: angeldlacruz_ alerta confirma convencion actual gobierno boric izquierda quieren expropiar robar ahorros
Tweet 2: juanpabloswett daniel stingo confirma berfontaine tenia razon heredable plata castellano robaran
------------------------------
Percentil 25:
Tweet 1: mister_wolf_0 udi canal libertad desarrollo elisa loncon tremenda elisaloncon enfrentar programa lleno
Tweet 2: elisaloncon razon rechazar gracias
------------------------------
Percentil 50:
Tweet 1: arturozunigaj derecha propuso bajar quorum reformas constitucionales via presidente udi javiermaca
Tweet 2: mister_wolf_0 rechazo caera mentiras partir ahora nuevo texto constituyente sera pilar derribar
------------------------------
Percentil 75:
Tweet 1: retirototalafp tere_marinovic explica paso paso expropiaran ahorros quintoretirourgente retirototalafp qui
Tweet 2: cgabriel01 tere_marinovic benbrereton18 esperanza
------------------------------
Percentil 95:
Tweet 1: kakaroto12345 eivor877 tere_marino

#### k=4

In [15]:
find_tweet_pairs(tweets_sample, 4, 0.05)

Calculando similitud: 100%|█████████| 499500/499500 [00:26<00:00, 19049.25par/s]

Percentil 0:
Tweet 1: angeldlacruz_ alerta confirma convencion actual gobierno boric izquierda quieren expropiar robar ahorros
Tweet 2: criordor llego dia acabo trabajo convencionconstitucional sentimos alegria logrado gracias todas
------------------------------
Percentil 25:
Tweet 1: angeldlacruz_ alerta confirma convencion actual gobierno boric izquierda quieren expropiar robar ahorros
Tweet 2: bdelamaza leguleyada tramposa grupo convencionales presento viernes comision copia exacta norma rechazada
------------------------------
Percentil 50:
Tweet 1: tere_marinovic sabe usted vota convencion constitucional respecto inmigracion ilegal aqui puede verlo
Tweet 2: 2018_enrique tere_marinovic habra hacer campana rechazo mejor jefa campana
------------------------------
Percentil 75:
Tweet 1: tere_marinovic presidente chile diciendole salga aca mujer espacio publico
Tweet 2: black_hat_ok gran parte ventaja rechazo debemos grandes mujeres tere_marinovic rocicantuarias
---------------------

Los resultados que más me hacen sentido son k=3 y s=0.1

# Locally Sensitive Hashing

In [16]:
from datasketch import MinHash, MinHashLSH
import numpy as np

In [17]:
def get_minhash(s):
    # Obtiene los k-shingles del texto
    shingle_set = get_k_shingles(s)

    # Crea un objeto MinHash con 16 permutaciones
    m = MinHash(num_perm=16)

    for s in shingle_set:
        m.update(s.encode('latin1'))

    return m

# Aplica la función get_minhash a cada tweet para obtener los MinHash
processed_tweets['minhash'] = processed_tweets['text'].progress_apply(get_minhash)

100%|███████████████████████████████| 4592806/4592806 [32:11<00:00, 2377.62it/s]


In [55]:
# Construye el índice LSH
lsh = MinHashLSH(threshold=0.5, num_perm=16)

In [56]:
# Inserta cada MinHash en el índice LSH
for i, mh in tqdm(processed_tweets['minhash'].items(), total=len(processed_tweets)):
    lsh.insert(str(i), mh)

100%|██████████████████████████████| 4592806/4592806 [01:10<00:00, 65197.99it/s]


# Resultados

In [89]:
def get_neighbors(processed_tweets, idx, s, k):
    # Lista para almacenar los resultados
    results = []
    
    # Obtiene el MinHash del tweet en el índice
    mh_idx = processed_tweets.loc[idx, 'minhash']

    # Encuentra los vecinos en el índice LSH
    lsh_keys = lsh.query(mh_idx)
    
    # Se obtiene el text
    text_idx = processed_tweets.loc[idx, 'text']
    
    # Para cada vecino
    for key in lsh_keys:
        # Obtiene el MinHash del vecino
        mh_neighbor = processed_tweets.loc[int(key), 'minhash']
        
        # Se obtiene el text del vecino
        text_neighbor = processed_tweets.loc[int(key), 'text']
        
        # Calcula la similitud de Jaccard
        jaccard_sim = jaccard_similarity(get_k_shingles(text_idx, k), get_k_shingles(text_neighbor, k))
        
        # Si la similitud de Jaccard supera el umbral s, guarda el ID del vecino y la similitud
        if jaccard_sim >= s:
            results.append((processed_tweets.loc[int(key), 'id'], jaccard_sim))
            
    return (processed_tweets.loc[idx, 'id'], results)

In [102]:
def print_percentiles(tweets, tweet_id, neighbors):
    # Obtiene los IDs y las similitudes de los vecinos
    neighbor_ids, sims = zip(*neighbors)
    
    # Convierte las similitudes en un array de NumPy para cálculos más fáciles
    sims = np.array(sims)
    
    # Calcula los percentiles
    p0 = np.percentile(sims, 0)
    p25 = np.percentile(sims, 10)
    p75 = np.percentile(sims, 20)
    p95 = np.percentile(sims, 95)
    
    # Imprime el tweet original
    print("Tweet original:", tweets[tweets['id'] == tweet_id]['text'].values[0])
    print("------------------------------")
    # Imprime los vecinos en cada percentil
    for p, percentile in zip([p0, p25, p75, p95], [0, 10, 20, 95]):
        # Encuentra el vecino más cercano al percentil actual
        closest_neighbor_id = neighbor_ids[np.argmin(np.abs(sims - p))]
        
        # Imprime el texto del vecino y la similitud
        print("\nVecino del percentil {} (similitud = {:.4f}):".format(percentile, p))
        print(tweets[tweets['id'] == closest_neighbor_id]['text'].values[0])
        print("------------------------------")

In [103]:
# Obteniendo los vecinos
idx_tweet, idx_neighbors = get_neighbors(processed_tweets, 1, 0.1, 3)

# Imprimiendo los percentiles
print_percentiles(tweets, idx_tweet, idx_neighbors)

Tweet original: RT @UTDTrabajoDigno: Mañana jueves a las 18hrs. comienza nuestro programa #DignificarelTrabajo en @uchileradio, con la Presidenta de @conve…
------------------------------

Vecino del percentil 0 (similitud = 0.1008):
Mañana en el Parque Quinta Normal. Vaya y comparta con nuestra gente. Lo esperamos!!! https://t.co/EyOkKPSxKy
------------------------------

Vecino del percentil 10 (similitud = 0.1371):
@BenitoBaranda @uchileradio Cara de ra…. Con las lucas de otros .. trabaja wn
------------------------------

Vecino del percentil 20 (similitud = 0.1517):
RT @MEQChile: Ahora sí, ¡ya estamos en Concepción!🥰 Comenzamos nuestras actividades en el Bío Bío con una entrevista en el programa "Nuestr…
------------------------------

Vecino del percentil 95 (similitud = 1.0000):
RT @UTDTrabajoDigno: Mañana jueves a las 18hrs. comienza nuestro programa #DignificarelTrabajo en @uchileradio, con la Presidenta de @conve…
------------------------------


In [104]:
# Obteniendo los vecinos
idx_tweet, idx_neighbors = get_neighbors(processed_tweets, 2_000_000, 0.1, 3)

# Imprimiendo los percentiles
print_percentiles(tweets, idx_tweet, idx_neighbors)

Tweet original: RT @24HorasTVN: 📃 #ElPaísQueQueremos  

@cretton15 :"Creo que la Convención Constitucional es uno de los organismos menos democráticos que…
------------------------------

Vecino del percentil 0 (similitud = 0.1000):
@patriciapolitz @convencioncl @sebastian_gray @KarolCariola Hasta el convencional curao habla weás más coherentes y reales que tus encuestas
------------------------------

Vecino del percentil 10 (similitud = 0.1060):
RT @PierreCurieD: Sería bueno que en la gira de la CC a Antofagasta, el Convencional @danielstingo les diga en su cara a los mineros que lo…
------------------------------

Vecino del percentil 20 (similitud = 0.1257):
RT @MEQChile: Hoy la #ConvenciónConstitucional sesionó en las Ruinas de Huanchaca, Antofagasta. Precioso lugar, un patrimonio lleno de hist…
------------------------------

Vecino del percentil 95 (similitud = 1.0000):
RT @24HorasTVN: 📃 #ElPaísQueQueremos  

@cretton15 :"Creo que la Convención Constitucional es uno de los organ